# Deep Learning Image Compression Project MLP part

This code applies predictive coding algoritm for lossless image compression with a basic MLP structure. Details of predictive coding algorithm can be found [here](https://web.stanford.edu/class/ee398a/handouts/lectures/06-Prediction.pdf)

The code has four parts

1. Huffman encoder (Coppied from [here](http://www.techrepublic.com/article/huffman-coding-in-python/))
2. Creation of prediction blocks and label for predictive coding
3. Linear regression algorithm for seeing the baseline
4. MLP algorithm (initial phase)

**PS:** I made all of the tests with a single image (lena412.bmp). JPEG-ls algorithm achieves 5.21 bpp (bits per pixel) with huffman coding, JPEG-2000-ls achieves ~4.31 bpp. MLP is able to achieve ~4.5 bpp

# Part-1: Huffman encoder and Entropy Calculation


In [1]:
#Binary tree data structure
#http://www.techrepublic.com/article/huffman-coding-in-python/
class Node(object):
	left = None
	right = None
	item = None
	weight = 0

	def __init__(self, i, w):
		self.item = i
		self.weight = w

	def setChildren(self, ln, rn):
		self.left = ln
		self.right = rn

	def __repr__(self):
		return "%s - %s — %s _ %s" % (self.item, self.weight, self.left, self.right)

	def __cmp__(self, a):
		return cmp(self.weight, a.weight)

In [2]:
#Huffman Encoder
#http://www.techrepublic.com/article/huffman-coding-in-python/

from itertools import groupby
from heapq import *


#Huffman encoder  
def huffman(input):
    itemqueue =  [Node(a,len(list(b))) for a,b in groupby(sorted(input))]
    heapify(itemqueue)
    while len(itemqueue) > 1:
        l = heappop(itemqueue)
        r = heappop(itemqueue)
        n = Node(None, r.weight+l.weight)
        n.setChildren(l,r)
        heappush(itemqueue, n) 
        
    codes = {}
    def codeIt(s, node):
        if node.item:
            if not s:
                codes[node.item] = "0"
            else:
                codes[node.item] = s
        else:
            codeIt(s+"0", node.left)
            codeIt(s+"1", node.right)
    codeIt("",itemqueue[0])
    return codes, "".join([codes[a] for a in input])


In [3]:
#Test Huffman encoder with an image

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
img=mpimg.imread('lena512.bmp')
#print(img.shape)
#imgplot=plt.imshow(img,cmap='gray')

img_input=img.reshape([-1]).astype(str)
#print(img_input)
huffman_img = huffman(img_input)
#print(huffman_img[1])

#print('Huffman code for ' + str(img) + ' is ' + str(huffman_img))
#print('Original length is '+str(len(input) * 8)+', length of huffman coding is '+ str(len(huffman(input)[1])))
print('Bitrate of the original image')
print('Bits per pixel is ' + str(float(len(huffman_img[1])/float(len(img_input)))) + ' bpp')

Bitrate of the original image
Bits per pixel is 7.46820831299 bpp


In [4]:
import numpy as np
def entropy(labels,degree):
    """ Computes entropy of label distribution. """
    n_labels = len(labels)

    if n_labels <= 1:
        return 0

    [counts, bins]  = np.histogram(labels,np.append(np.unique(labels),np.inf))
    #print(bins)
    probs = counts.astype(float) / float(n_labels)
    #print(probs)
    #n_classes = np.count_nonzero(probs)
    
    #if n_classes <= 1:
        #return 0
    

    # Compute standard entropy.
    if(degree==1):
        ent = -np.sum(np.multiply(probs,np.log2(probs)))
    else:
        ent = np.log2(np.sum(np.power(probs,degree)))/(1-degree)
    
    '''ent = 0.
    for i in probs:
        if(degree==1):
            ent -= i * np.log2(i)
        else:
            ent -= np.log2(np.sum(np.power()))'''

    return ent

In [12]:
#test
x=[1.,1.,1.,1.,1.,1.,1.,1.,-7.,-7.,-7.,-7.,-7.,100.,100.,100.,-4.,-4.,-4.,5.]
x=x+np.random.rand(20,1)
x=np.asarray(x)
#print(x)

print(entropy(x,1))
print(entropy(x,2))

41.319146419
-2.43295940728


# Part-2: Creation of prediction blocks and label for predictive coding


In [13]:
#Lossless image copmpression using predictive coding. For reference see below
#(https://web.stanford.edu/class/ee398a/handouts/lectures/06-Prediction.pdf)

from itertools import product


#Returns prediction blocks and the corresponding pixels in the image
#Very naive implementation, neglects boundaries, can be improved further
def pred_vectors(img,pred_size):
    (n,m)=img.shape #image size
    k,l=pred_size #Size of the predictive window
    
    fvec=np.zeros([(n-k-1)*(m-2*l),2*k*l+k+l])
    #print(fvec.shape)
    label = np.zeros([(n-k-1)*(m-2*l),1])
    for (i,j) in product(range(k,n-1), range(l,m-l)):
        #print(i,j)
        idx = (i-k)*(m-2*l)+j-l
        fvec_current =img[i-k:i,j-l:j+l+1].reshape([-1])
        fvec_current = np.append(fvec_current,img[i,j-l:j].reshape([-1]))
        fvec[idx,:]=fvec_current
        label[idx]=img[i,j]
        
    return fvec, label



fvec,label = pred_vectors(img,[3,7])

# Part-3: Linear regression algorithm for seeing the baseline


In [14]:
#First trial: Simple regression network. No relation to deep learning just to gain some intuition


from sklearn import datasets, linear_model


#Create the regression model using sklearn
regr = linear_model.LinearRegression()
regr.fit(fvec, label)

#Predict and quantize the labels
label_pred = np.round(regr.predict(fvec))

#Calculate the error
err=label_pred-label;

print('Results with linear regression')
#MSE
print('MSE is '  + str(np.mean(err**2)))

#Calculate Huffman coding of the error
huffman_err = huffman(err.reshape([-1]).astype(str))
print('Bits per pixel is ' + str(float(len(huffman_err[1])/float(len(err)))) + ' bpp')


Results with linear regression
MSE is 33.4797180849
Bits per pixel is 4.43469547481 bpp


# Part-4: MLP algorithm (initial phase)

In [28]:
#Second trial: MLP

import tensorflow as tf

def mlp(x, hidden_sizes, activation_fn=tf.nn.relu,dropout_rate=1.0,std_dev=1.0):
    if not isinstance(hidden_sizes, (list, tuple)):
        raise ValueError("hidden_sizes must be a list or a tuple")
    scope_args = {'initializer': tf.random_normal_initializer(stddev=std_dev)}
    for k in range(len(hidden_sizes)-1):
        layer_name="weights"+str(k)
        #FC layers
        with tf.variable_scope(layer_name, **scope_args):
            W = tf.get_variable('W', shape=[x.shape[-1], hidden_sizes[k]])
            b = tf.get_variable('b', shape=[hidden_sizes[k]])
            x = activation_fn(tf.matmul(x, W) + b)
            #Dropout before the last layer
            x = tf.nn.dropout(x, keep_prob=dropout_rate)
    #Softmax layer
    with tf.variable_scope('outlayer', **scope_args):
        W = tf.get_variable('W', shape=[x.shape[-1], hidden_sizes[-1]])
        b = tf.get_variable('b', shape=[hidden_sizes[-1]])
        return tf.matmul(x, W) + b
    


In [16]:
##Test for Renyi's Entropy

from pprint import pprint
#x=np.array([[1.,1.,1.,1.,1.,1.,1.,1.,2.,2.,2.,2.,2.,3.,3.,3.,4.,4.,5.,5.]])
n=9
x=np.array([[1,2,3,4,5,6,7,8,9],[1,3,28,10,23,12,43,1,9],[-4,0,2,1,54,1,3,8,-4]])
x=x.astype(np.float32)

c=np.power(n,-.2)
print(x.shape)
err_ = tf.placeholder(tf.float32, [None, n])
y_ = tf.placeholder(tf.float32, [None, 1])

#x_std=tf.reduce_mean(x_)
x_mean,x_std=tf.nn.moments(err_,axes=[1])
x_std=tf.sqrt(tf.multiply(x_std,float(n)/float(n-1)))
h_=tf.reshape(tf.multiply(1.06,tf.multiply(x_std,c)),[-1,1])



#prob_mat1=tf.div(err_,tf.matmul(h_,np.ones([1,n]).astype(np.float32)))
#Calculate Entropy
prob_mat = tf.exp(tf.multiply(-.5,tf.pow(tf.div((err_+50),tf.matmul(h_,np.ones([1,n]).astype(np.float32))),2)))
prob_ = tf.multiply((1/(np.sqrt(2*np.pi)*n*tf.transpose(h_))),tf.reduce_sum(prob_mat,axis=[1]))
#Calculate Entropy
sum_for_ent=tf.pow(prob_,2)


for k in range(1,100):
    #print(k)
    prob_mat = tf.exp(tf.multiply(-.5,tf.pow(tf.div((err_-k+50),tf.matmul(h_,np.ones([1,n]).astype(np.float32))),2)))
    prob_ = tf.multiply((1/(np.sqrt(2*np.pi)*n*tf.transpose(h_))),tf.reduce_sum(prob_mat,axis=[1]))
    #Calculate Entropy
    sum_for_ent=sum_for_ent+tf.pow(prob_,2)
renyi_ent2=-tf.log(sum_for_ent)#/tf.log(2.)
loss=tf.reduce_mean(renyi_ent2)

with tf.Session() as sess:
        # that is how we "execute" statements 
        # (return None, e.g. init() or train_op())
        # or compute parts of graph defined above (loss, output, etc.)
        # given certain input (x_, y_)
    #sess.run(tf.global_variables_initializer())
    f_val=sess.run(renyi_ent2, feed_dict={err_:x})
    f_val2=sess.run(loss, feed_dict={err_:x})
    pprint(f_val)
    pprint(f_val2)

(3, 9)
array([[ 2.46076798,  4.04637003,  4.04785681]], dtype=float32)
3.5183315


In [18]:
pprint(fvec_n.shape)

(252984, 52)


In [41]:
##DOES NOT WORK!!!!!!!!!!!
#TRIAL WITH RENYIS ENTROPY
#Normalize the vectors and labels
#Sometimes does not work beacuse of wron initialization

fvec_n=fvec/np.round(np.max(label))
label_n = label/np.round(np.max(label))

n=fvec_n.shape[1]
def test_classification(model_function, learning_rate=0.1):

    with tf.Graph().as_default() as g:
        # where are you going to allocate memory and perform computations
        with tf.device("/gpu:0"):
            
            # define model "input placeholders", i.e. variables that are
            # going to be substituted with input data on train/test time
            x_ = tf.placeholder(tf.float32, [None, fvec_n.shape[1]])
            y_ = tf.placeholder(tf.float32, [None, 1])
            y_logits = model_function(x_)
            
            # naive implementation of loss:
            # > losses = y_ * tf.log(tf.nn.softmax(y_logits))
            # > tf.reduce_mean(-tf.reduce_sum(losses, 1))
            # can be numerically unstable.
            #
            # so here we use tf.nn.softmax_cross_entropy_with_logits on the raw
            # outputs of 'y', and then average across the batch.
            
            #Basic MSE loss
            loss = tf.reduce_mean(tf.pow(tf.subtract(y_,y_logits), 2.0))
            
            #RENYI ENTROPY IS NOT DIFFERENTIABLE
            #kernel = tf.contrib.distributions.Normal(mu=0., sigma=1.)
            #loss = tf.contrib.bayesflow.entropy.entropy_shannon(p=kernel,z=tf.subtract(y_,y_logits))
            #loss = tf.reduce_mean(tf.abs(tf.subtract(y_,y_logits)))
            """
            err_=tf.subtract(y_,y_logits)
            x_mean,x_std=tf.nn.moments(err_,axes=[1])
            x_std=tf.sqrt(tf.multiply(x_std,float(n)/float(n-1)))
            h_=tf.reshape(tf.multiply(1.06,tf.multiply((x_std+.001),c)),[-1,1])



            #prob_mat1=tf.div(err_,tf.matmul(h_,np.ones([1,n]).astype(np.float32)))
            #Calculate Entropy
            prob_mat = tf.exp(tf.multiply(-.5,tf.pow(tf.div((err_+50),tf.matmul(h_,np.ones([1,n]).astype(np.float32))),2)))
            prob_ = tf.multiply((1/(np.sqrt(2*np.pi)*n*tf.transpose(h_))),tf.reduce_sum(prob_mat,axis=[1]))
            #Calculate Entropy
            sum_for_ent=tf.pow(prob_,2)


            for k in range(1,100):
                #print(k)
                prob_mat = tf.exp(tf.multiply(-.5,tf.pow(tf.div((err_-k+50),tf.matmul(h_,np.ones([1,n]).astype(np.float32))),2)))
                prob_ = tf.multiply((1/(np.sqrt(2*np.pi)*n*tf.transpose(h_))),tf.reduce_sum(prob_mat,axis=[1]))
                #Calculate Entropy
                sum_for_ent=sum_for_ent+tf.pow(prob_,2)
            renyi_ent2=-tf.log(sum_for_ent)#/tf.log(2.)
            loss=tf.reduce_mean(renyi_ent2)
            """
            
            #train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
            train_step = tf.train.AdamOptimizer(learning_rate=5e-3,beta1=0.3,beta2=0.999, 
                                                epsilon=1e-08,use_locking=False).minimize(loss)
           
            y_pred = y_logits
            correct_prediction = tf.equal(y_pred, y_)
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    with g.as_default(), tf.Session() as sess:
        # that is how we "execute" statements 
        # (return None, e.g. init() or train_op())
        # or compute parts of graph defined above (loss, output, etc.)
        # given certain input (x_, y_)
        sess.run(tf.global_variables_initializer())
        #sess.run(tf.global_variables_initializer())
        
        # train
        #print(label.shape[0])
        batch_size=100
        ids=[i for i in range(batch_size)]
        for iter_i in range(100001):
            #print(iter_i)
            #print(label.shape[0])
            #print(2*my_range)
            batch_xs = fvec_n[ids,:] 
            batch_ys = label_n[ids]
            ids=[(ids[0]+batch_size+i)%label.shape[0] for i in range(batch_size)]
            tf_feed_dict = {x_: batch_xs, y_: batch_ys}
            '''
            acc_value = sess.run(h_, feed_dict=tf_feed_dict)
            acc_value2 = sess.run(y_logits, feed_dict=tf_feed_dict)
            acc_value3 = sess.run(prob_mat, feed_dict=tf_feed_dict)
            y_pred_val = sess.run(y_pred, feed_dict=tf_feed_dict)
            err_value = np.round((sess.run(y_pred, feed_dict=tf_feed_dict)-batch_ys))
            print(acc_value)
            print(acc_value2)
            print(acc_value3)
            ''' 
            sess.run(train_step, feed_dict={x_: batch_xs, y_: batch_ys})
            
            # test trained model
            if iter_i % 2000 == 0:
                #tf_feed_dict = {x_: fvec_n, y_: label_n}
                tf_feed_dict = {x_: batch_xs, y_: batch_ys}
                acc_value = sess.run(loss, feed_dict=tf_feed_dict)
                y_pred_val = sess.run(y_pred, feed_dict=tf_feed_dict)
                err_value = np.round((sess.run(y_pred, feed_dict=tf_feed_dict)-batch_ys)*255)
                '''
                acc_value2 = sess.run(y_logits, feed_dict=tf_feed_dict)
                acc_value3 = sess.run(prob_mat, feed_dict=tf_feed_dict)
                y_pred_val = sess.run(y_pred, feed_dict=tf_feed_dict)
                err_value = np.round((sess.run(y_pred, feed_dict=tf_feed_dict)-batch_ys))
                print(acc_value)
                print(acc_value2)
                print(acc_value3)
                print(err_value.reshape([-1]))
                '''
                entropy1_err = entropy(err_value.reshape([-1]),1)
                entropy2_err = entropy(err_value.reshape([-1]),2)
                huffman_err = huffman(err_value.reshape([-1]).astype(str))
                huffman_bpp = float(len(huffman_err[1])/float(len(err_value)))
                print('iteration %d\t mse loss: %.5f\t entropy1: %.5f\t entropy2: %.5f\t Huffman bitrate is %.3f'%
                      (iter_i, acc_value, entropy1_err,entropy2_err,huffman_bpp))
        #err_value =  np.round((sess.run(y_pred, feed_dict=tf_feed_dict)-label_n)*255)
        #print(err_value)
                
test_classification(lambda x: mlp(x, [32,16,8,4,2,1], activation_fn=tf.nn.relu,std_dev=1e-1), learning_rate=0.05)

iteration 0	 mse loss: 0.19890	 entropy1: 5.04256	 entropy2: 4.80390	 Huffman bitrate is 5.080
iteration 2000	 mse loss: 0.05796	 entropy1: 5.88511	 entropy2: 5.70275	 Huffman bitrate is 5.950
iteration 4000	 mse loss: 0.06422	 entropy1: 5.74021	 entropy2: 5.35698	 Huffman bitrate is 5.810
iteration 6000	 mse loss: 0.02628	 entropy1: 5.03464	 entropy2: 4.62950	 Huffman bitrate is 5.070
iteration 8000	 mse loss: 0.07235	 entropy1: 5.61085	 entropy2: 5.31043	 Huffman bitrate is 5.660
iteration 10000	 mse loss: 0.04673	 entropy1: 5.81101	 entropy2: 5.64386	 Huffman bitrate is 5.870
iteration 12000	 mse loss: 0.03350	 entropy1: 5.36837	 entropy2: 5.02093	 Huffman bitrate is 5.420
iteration 14000	 mse loss: 0.05526	 entropy1: 6.02511	 entropy2: 5.77992	 Huffman bitrate is 6.090
iteration 16000	 mse loss: 0.02354	 entropy1: 4.91027	 entropy2: 4.48681	 Huffman bitrate is 4.960
iteration 18000	 mse loss: 0.02371	 entropy1: 5.12047	 entropy2: 4.82011	 Huffman bitrate is 5.170
iteration 20000	 m

KeyboardInterrupt: 

In [42]:
##WORKING CODE WITH MSE
#Normalize the vectors and labels
#Sometimes does not work beacuse of wron initialization

fvec_n=fvec/np.round(np.max(label))
label_n = label/np.round(np.max(label))
def test_classification(model_function, learning_rate=0.1):

    with tf.Graph().as_default() as g:
        # where are you going to allocate memory and perform computations
        with tf.device("/cpu:0"):
            
            # define model "input placeholders", i.e. variables that are
            # going to be substituted with input data on train/test time
            x_ = tf.placeholder(tf.float32, [None, fvec_n.shape[1]])
            y_ = tf.placeholder(tf.float32, [None, 1])
            y_logits = model_function(x_)
            
            # naive implementation of loss:
            # > losses = y_ * tf.log(tf.nn.softmax(y_logits))
            # > tf.reduce_mean(-tf.reduce_sum(losses, 1))
            # can be numerically unstable.
            #
            # so here we use tf.nn.softmax_cross_entropy_with_logits on the raw
            # outputs of 'y', and then average across the batch.
            
            #Basic MSE loss
            #loss = tf.reduce_mean(tf.pow(tf.subtract(y_,y_logits), 2.0))
            
            #SHANNON ENTROPY IS NOT DIFFERENTIABLE
            #kernel = tf.contrib.distributions.Normal(mu=0., sigma=1.)
            #loss = tf.contrib.bayesflow.entropy.entropy_shannon(p=kernel,z=tf.subtract(y_,y_logits))
            loss = tf.reduce_mean(tf.abs(tf.subtract(y_,y_logits)))
            #train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
            train_step = tf.train.AdamOptimizer(learning_rate=5e-3,beta1=0.3,beta2=0.999, 
                                                epsilon=1e-08,use_locking=False).minimize(loss)
           
            y_pred = y_logits
            correct_prediction = tf.equal(y_pred, y_)
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    with g.as_default(), tf.Session() as sess:
        # that is how we "execute" statements 
        # (return None, e.g. init() or train_op())
        # or compute parts of graph defined above (loss, output, etc.)
        # given certain input (x_, y_)
        sess.run(tf.global_variables_initializer())
        #sess.run(tf.global_variables_initializer())
        
        # train
        #print(label.shape[0])
        ids=[i for i in range(100)]
        for iter_i in range(100001):
            #print(label.shape[0])
            #print(2*my_range)
            batch_xs = fvec_n[ids,:] 
            batch_ys = label_n[ids]
            ids=[(ids[0]+100+i)%label.shape[0] for i in range(100)]
            sess.run(train_step, feed_dict={x_: batch_xs, y_: batch_ys})
            
            # test trained model
            if iter_i % 2000 == 0:
                tf_feed_dict = {x_: fvec_n, y_: label_n}
                acc_value = sess.run(loss, feed_dict=tf_feed_dict)
                y_pred_val = sess.run(y_pred, feed_dict=tf_feed_dict)
                err_value = np.round((sess.run(y_pred, feed_dict=tf_feed_dict)-label_n)*255)
                #print(err_value.reshape([-1]))
                entropy1_err = entropy(err_value.reshape([-1]),1)
                entropy2_err = entropy(err_value.reshape([-1]),2)
                huffman_err = huffman(err_value.reshape([-1]).astype(str))
                huffman_bpp = float(len(huffman_err[1])/float(len(err_value)))
                print('iteration %d\t mse loss: %.5f\t entropy1: %.5f\t entropy2: %.5f\t Huffman bitrate is %.3f'%
                      (iter_i, acc_value, entropy1_err,entropy2_err,huffman_bpp))
        err_value =  np.round((sess.run(y_pred, feed_dict=tf_feed_dict)-label_n)*255)
        #print(err_value)
                
test_classification(lambda x: mlp(x, [32,16,8,4,2,1], activation_fn=tf.nn.relu,std_dev=1e-1), learning_rate=0.05)

iteration 0	 mse loss: 0.63563	 entropy1: 7.44656	 entropy2: 7.32115	 Huffman bitrate is 7.470
iteration 2000	 mse loss: 0.03646	 entropy1: 5.11033	 entropy2: 4.50077	 Huffman bitrate is 5.160
iteration 4000	 mse loss: 0.03272	 entropy1: 5.14986	 entropy2: 4.64941	 Huffman bitrate is 5.206
iteration 6000	 mse loss: 0.02798	 entropy1: 4.86304	 entropy2: 4.31970	 Huffman bitrate is 4.937
iteration 8000	 mse loss: 0.01996	 entropy1: 4.70085	 entropy2: 4.13966	 Huffman bitrate is 4.826
iteration 10000	 mse loss: 0.02057	 entropy1: 4.63546	 entropy2: 4.08668	 Huffman bitrate is 4.750
iteration 12000	 mse loss: 0.02396	 entropy1: 4.83564	 entropy2: 4.39802	 Huffman bitrate is 4.915
iteration 14000	 mse loss: 0.01724	 entropy1: 4.54542	 entropy2: 4.01856	 Huffman bitrate is 4.687
iteration 16000	 mse loss: 0.02156	 entropy1: 4.61002	 entropy2: 4.05941	 Huffman bitrate is 4.710
iteration 18000	 mse loss: 0.01809	 entropy1: 4.57681	 entropy2: 4.03880	 Huffman bitrate is 4.713
iteration 20000	 m

KeyboardInterrupt: 